# Load stuff

In [1]:
!pip install -Uqq zss conllu tabulate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

from sklearn.cluster import HDBSCAN
import conllu
import json

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def corpus_from_file(filename):
    corpus = pd.read_csv(filename, sep='\t')
    corpus.drop('feats', axis=1, inplace=True)
    corpus.drop('xpos', axis=1, inplace=True)
    return corpus

dev = corpus_from_file('short_dev.csv')
dev

,sent_id,token_id,form,lemma,upos,head_id,deprel,head_dist
0,0,38,و,w,CCONJ,39,cc,1
1,0,39,يبلغ,balag-u_1,VERB,34,root,0
2,0,40,300,DEFAULT,NUM,42,nummod,2
3,0,41,الف,>alof_1,NUM,40,compound,-1
4,0,42,دولار,duwlAr_1,NOUN,39,obj,-3
...,...,...,...,...,...,...,...,...
1295,259,18,ينفذ,naf~a*_1,VERB,17,root,0
1296,259,19,ها,hA,PRON,18,obj,-1
1297,259,20,الشباب,$ab~_1,NOUN,18,nsubj,-2
1298,259,21,في,fiy_1,ADP,22,case,1


# Tree Distance

In [3]:
from zss import simple_distance, Node

A = (
    Node("f")
        .addkid(Node("a")
            .addkid(Node("h"))
            .addkid(Node("c")
                .addkid(Node("l"))))
        .addkid(Node("e"))
    )
B = (
    Node("f")
        .addkid(Node("a")
            .addkid(Node("d"))
            .addkid(Node("c")
                .addkid(Node("b"))))
        .addkid(Node("e"))
    )
assert simple_distance(A, B) == 2

# Sentence to Tree

In [41]:
def print_tree(node, depth=0):
    print(f"{'|   ' * depth}{node.label}")
    for ch in node.children:
        print_tree(ch, depth+1)

def corpus_get_sentence(corpus, sent_id):
    return corpus.loc[corpus['sent_id'] == sent_id].reset_index()
        
def sentence_to_tree(sent):
    def _sentence_to_tree(index):
        token = sent.loc[index]
        graph_node = Node(token['upos'])
        # print(sent_node)
        # sent_node_id = sent_node['token_id'].item()
        # print(sent_node_id)
        for ch in sent.index[sent['head_id'] == token['token_id']].tolist():
            graph_node.addkid(_sentence_to_tree(ch))
        return graph_node

    return _sentence_to_tree(sent.index[sent['head_dist'] == 0][0])

print(corpus_get_sentence(dev, 0))
print_tree(sentence_to_tree(corpus_get_sentence(dev, 0)))

   index  sent_id  token_id   form      lemma   upos  head_id    deprel  head_dist
0      0        0        38      و          w  CCONJ       39        cc          1
1      1        0        39   يبلغ  balag-u_1   VERB       34      root          0
2      2        0        40    300    DEFAULT    NUM       42    nummod          2
3      3        0        41    الف    >alof_1    NUM       40  compound         -1
4      4        0        42  دولار   duwlAr_1   NOUN       39       obj         -3
VERB
|   CCONJ
|   NOUN
|   |   NUM
|   |   |   NUM


# Sentence Distance

In [44]:
def sentence_distance(sent1, sent2):
    return simple_distance(
        sentence_to_tree(sent1),
        sentence_to_tree(sent2)
    )

print(corpus_get_sentence(dev, 0))
print(corpus_get_sentence(dev, 1))
sentence_distance(
    corpus_get_sentence(dev, 0),
    corpus_get_sentence(dev, 1))

   index  sent_id  token_id   form      lemma   upos  head_id    deprel  head_dist
0      0        0        38      و          w  CCONJ       39        cc          1
1      1        0        39   يبلغ  balag-u_1   VERB       34      root          0
2      2        0        40    300    DEFAULT    NUM       42    nummod          2
3      3        0        41    الف    >alof_1    NUM       40  compound         -1
4      4        0        42  دولار   duwlAr_1   NOUN       39       obj         -3
   index  sent_id  token_id   form      lemma   upos  head_id deprel  head_dist
0      5        1         8      و          w  CCONJ       10   iobj          2
1      6        1         9     هو     huwa_1   PRON       10  nsubj          1
2      7        1        10   يصعد  SaEid-a_1   VERB        2   root          0
3      8        1        11    الى    <ilaY_1    ADP       12   case          1
4      9        1        12  الباص      bAS_1   NOUN       10    obj         -2


3.0

# Cluster Sentences Based on Similarity Matrix

In [48]:
from tqdm.notebook import tqdm

def corpus_first_n_sentences(corpus, N):
    return corpus.loc[corpus['sent_id'].isin(corpus['sent_id'].unique()[:N])]

def corpus_distance_matrix(corpus):
    sent_ids = corpus['sent_id'].unique()
    N = len(sent_ids)
    dist = np.zeros((N, N))
    for i in tqdm(range(N)):
        for j in tqdm(range(i, N), leave=False):
            dist[i][j] = dist[j][i] = sentence_distance(
                corpus_get_sentence(corpus, sent_ids[i]),
                corpus_get_sentence(corpus, sent_ids[j]))
    return sent_ids, dist

corpus_distance_matrix(corpus_first_n_sentences(dev, 10))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([[0., 3., 4., 4., 3., 4., 4., 4., 4., 4.],
        [3., 0., 5., 2., 1., 3., 4., 3., 2., 3.],
        [4., 5., 0., 6., 5., 4., 3., 6., 5., 4.],
        [4., 2., 6., 0., 3., 4., 4., 4., 3., 4.],
        [3., 1., 5., 3., 0., 2., 4., 2., 3., 2.],
        [4., 3., 4., 4., 2., 0., 4., 3., 4., 2.],
        [4., 4., 3., 4., 4., 4., 0., 5., 4., 3.],
        [4., 3., 6., 4., 2., 3., 5., 0., 4., 3.],
        [4., 2., 5., 3., 3., 4., 4., 4., 0., 4.],
        [4., 3., 4., 4., 2., 2., 3., 3., 4., 0.]]))

In [52]:
def cluster_corpus(corpus, matrix_fn, min_cluster_size=5):
    print('== Computing distance matrix...')
    %time
    sent_ids, matrix = matrix_fn(corpus)
    print('== Clustering...')
    %time
    hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size).fit(matrix)
    print(hdb.labels_)
    print(hdb.probabilities_)

    print('== Aggregating...')
    from collections import defaultdict
    clusters = defaultdict(lambda: [])
    cluster_prob = defaultdict(lambda: 1)
    for prob, sent_id, label in zip(hdb.probabilities_, sent_ids, hdb.labels_):
        clusters[label].append(sent_id)
        cluster_prob[label] *= prob
    print()
    return sorted([(cluster_prob[k], k, v) for k, v in clusters.items()], reverse=True)

def clusters_write_to_file(clusters, corpus, filename):
    print('== Writing to file...')
    with open(filename, 'w') as f:
        for p, label, v in clusters:
            if label >= 0:
                f.write(f"\nCluster {label:02d}:\n")
            else:
                f.write(f"\n\n\nNOT CLUSTERED:\n")
            f.write(f"\t%{p*100:.2f}\n")
            for sent_id in v:
                f.write(' '.join(corpus_get_sentence(corpus, sent_id)['form']))
                f.write('\n')

dev_small = corpus_first_n_sentences(dev, 40)
clusters = cluster_corpus(dev_small, corpus_distance_matrix, 2)
clusters_write_to_file(clusters, dev_small, 'trees_cluster.txt')

== Computing distance matrix...
CPU times: user 1 μs, sys: 1e+03 ns, total: 2 μs
Wall time: 4.05 μs


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

== Clustering...
CPU times: user 3 μs, sys: 1 μs, total: 4 μs
Wall time: 7.15 μs
[ 1  7  2  8  7  9  4 10 10 -1 11 10  1 -1  3 -1  0  0  8 11  5  6  3  4
  6  4  6  5 -1 -1  9  7 11 11 -1  5  2  7  5  6]
[1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.5 1.  1.  0.  1.  0.  1.  1.
 1.  1.  1.  0.5 1.  1.  0.5 0.  1.  1.  0.  0.  1.  1.  1.  1.  0.  1.
 1.  1.  1.  1. ]
== Aggregating...

== Writing to file...
